<a href="https://colab.research.google.com/github/helloisachinaide/ProjetoAPI4/blob/main/api_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Nova seção

In [ ]:
import pandas as pd
import datetime
import glob

# Defina o caminho do arquivo .txt
txt_file = '2023Carga.txt'

# Ler o arquivo .txt e exibir os primeiros registros para verificar a leitura
df = pd.read_csv(txt_file, delimiter=';', encoding='utf-8')

# Exibir as primeiras linhas do DataFrame para verificar se foi lido corretamente
print("Visualização dos dados lidos do arquivo .txt:")
print(df.head())

# Salvar o arquivo como .csv, caso o DataFrame não esteja vazio
if not df.empty:
    df.to_csv('2023Carga.csv', encoding='utf-8', index=False)
    print("Arquivo convertido de .txt para .csv com sucesso!")
else:
    print("Erro: O DataFrame está vazio após a leitura do arquivo .txt")

# Verificar se o CSV gerado está correto
df_check = pd.read_csv(txt_file, encoding='utf-8')
print("Visualização do arquivo CSV gerado:")
print(df_check.head())

In [ ]:
#Excluir dados desnecessarios

df = pd.read_csv('2023Carga.csv')

# Excluir colunas indesejadas
colunas_a_excluir = ['Carga Geral Acondicionamento', 'ConteinerEstado', 'FlagAutorizacao', 'FlagCabotagem', 'FlagCabotagemMovimentacao', 'FlagConteinerTamanho', 'FlagLongoCurso', 'FlagMCOperacaoCarga', 'FlagOffshore', 'FlagTransporteViaInterioir']  # Substitua pelos nomes das colunas que você deseja excluir
df = df.drop(columns=colunas_a_excluir)

# Exibir as colunas após a exclusão
print("Colunas após a exclusão:", df.columns)

# Salvar o DataFrame modificado de volta em um novo arquivo CSV
df.to_csv('2023Carga_NOVO.csv', index=False)

In [ ]:
#excluir dados nulos

df = pd.read_csv('2023Atracacao.csv')

# Excluir linhas onde 'Data Chegada' é nula
df = df.dropna(subset=['Data Chegada'])

# Exibir as primeiras linhas após a exclusão
print("Após a exclusão:")
print(df.head())

# Salvar o DataFrame modificado em um novo arquivo CSV
df.to_csv('2023Atracacao_mod.csv', index=False)

In [ ]:

#Juntar todas as tabelas de atracacao e carga de 2014-2023 em dois CSV's


# Encontrar todos os arquivos .csv no diretório atual
arquivos = glob.glob('*.csv')

# Criando uma lista para armazenar os DataFrames
dataframes = []

# Lendo apenas os arquivos de atracação (mod) ou carga(Carga)
for arquivo in arquivos:
    if 'mod' in arquivo:  # Verifica se o nome contém 'Carga' ou 'mod'
        try:
            df = pd.read_csv(arquivo)
            dataframes.append(df)
        except Exception as e:
            print(f"Erro ao ler {arquivo}: {e}")

# Verifica se há DataFrames para concatenar
if dataframes:
    # Concatenando todos os DataFrames de atracação ou carga
    resultado = pd.concat(dataframes, ignore_index=True)

    # Salvando o resultado em um novo arquivo .csv
    resultado.to_csv('arquivo_atracacao_final_NOVO.csv', index=False)
    print("Arquivos de atracação unidos com sucesso!")
else:

    print("Nenhum arquivo de atracação válido encontrado.")

In [ ]:
#filtrar a tabela de TODAS as cargas para somente SH4 1005 = milho


data = pd.read_csv('arquivo_carga_final.csv')


# Filtrar apenas os registros do SH4 1005
filtered_data = data[data['CDMercadoria'] == '1005']

# Salvar os dados filtrados em um novo arquivo CSV
filtered_data.to_csv('arquivo_carga_final_(so_milho).csv', index=False)

In [ ]:
#converte datas para novo padrão

ANTAQ_Limpa = pd.read_csv('arquivo_atracacao_final_NOVO.csv', low_memory=False)

colunas_data = [
    'Data Chegada', 'Data Atracação', 'Data Início Operação',
    'Data Término Operação', 'Data Desatracação'
]

for coluna in colunas_data:
    ANTAQ_Limpa[coluna] = pd.to_datetime(ANTAQ_Limpa[coluna], format='%d/%m/%Y %H:%M:%S', errors='coerce')

# As colunas agora estão no formato datetime
print("Colunas de data convertidas com sucesso.")
ANTAQ_Limpa.to_csv('arquivo_atracacao_final_NOVO.csv', index=False)

In [ ]:
#criaçao de novas colunas fazendo calculos pertinentes

ANTAQ_Limpa['Tempo_espera_chegada'] = (ANTAQ_Limpa['Data Atracação'] - ANTAQ_Limpa['Data Chegada']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_espera_inicial'] = (ANTAQ_Limpa['Data Início Operação'] - ANTAQ_Limpa['Data Atracação']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_operacao'] = (ANTAQ_Limpa['Data Término Operação'] - ANTAQ_Limpa['Data Início Operação']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_espera_final'] = (ANTAQ_Limpa['Data Desatracação'] - ANTAQ_Limpa['Data Término Operação']).dt.total_seconds() / 3600
ANTAQ_Limpa['Tempo_ocupacao'] = (ANTAQ_Limpa['Data Desatracação'] - ANTAQ_Limpa['Data Atracação']).dt.total_seconds() / 3600

print("Colunas novas criadas com sucesso.")
ANTAQ_Limpa.to_csv('arquivo_atracacao_final_NOVO.csv', index=False)

In [ ]:
#concatenaçao dos dados, juntando tudo em um CSV

tabela1 = pd.read_csv('arquivo_atracacao_final_NOVO.csv', low_memory=False)
tabela2 = pd.read_csv('arquivo_carga_final_(so_milho).csv', low_memory=False)



resultado = pd.merge(tabela1, tabela2, on='IDAtracacao', how='inner')


pd.DataFrame(resultado)
resultado.to_csv('carga_&_atracacao(so_milho)_NOVO.csv', index=False)
print('dados juntados')
print(resultado)

In [ ]:
#conferencia se ha dados duplicados depois de contatenar, se tiver são excluidos

ANTAQ_Limpa = pd.read_csv('carga_&_atracacao(so_milho).csv', low_memory=False)

# Identificar duplicados
duplicados = ANTAQ_Limpa['IDAtracacao'].duplicated(keep=False)

# Excluir todas as linhas que têm valores duplicados
ANTAQ_Limpa = ANTAQ_Limpa[~duplicados]

# Exibir o DataFrame atualizado
print("Tabela atualizada sem duplicatas na coluna IDAtracacao:")
print(ANTAQ_Limpa)



ANTAQ_Limpa.to_csv('carga_&_atracacao_NOVO.csv', index=False)

In [ ]:
#arquivo final: carga_&_atracacao_NOVO.csv